<a href="https://colab.research.google.com/github/lauracerro/Curso2019-2020/blob/master/IREI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **PRUEBAS USANDO LIGHT GRADIENT BOOSTING MACHINE**








## Preprocessing

In [7]:
import pandas as pd
pd.options.display.max_columns = None
import lightgbm as lgb
gbm = lgb.LGBMRanker()

We load the CSV for training (which is ordered) and split the relevancy from the other fields

In [10]:
data = pd.read_csv('/content/GlucoseInBlood.csv', delimiter = ';')
data


,loinc_num,long_common_name,component,system,property,relevancy
0,14749-6,Glucose [Moles/volume] in Serum or Plasma,Glucose,Ser/Plas,SCnc,5
1,14764-5,Glucose [Moles/volume] in Serum or Plasma --3 ...,Glucose^3H post 100 g glucose PO,Ser/Plas,SCnc,5
2,35184-1,Fasting glucose [Mass or Moles/volume] in Seru...,Glucose^post CFst,Ser/Plas,MSCnc,5
3,74774-1,Glucose [Mass/volume] in Serum Plasma or Blood,Glucose,Ser/Plas/Bld,MCnc,5
4,14747-0,Glucose [Moles/volume] in Pleural fluid,Glucose,Plr fld,SCnc,4
...,...,...,...,...,...,...
62,31100-1,Hematocrit [Volume Fraction] of Blood by Imped...,Hematocrit,Bld,VFr,1
63,33870-7,Bilirubin.total [Presence] in Unspecified spec...,Bilirubin,XXX,PrThr,1
64,4671-4,Protein C [Mass/volume] in Plasma,Protein C,Plas,MCnc,1
65,54439-5,Calcium bilirubinate/Total in Stone,Calcium bilirubinate/Total,Calculus,MFr,1


In [11]:
y_train= data.relevancy
X_train= data.drop('relevancy',axis=1)
y_train

0     5
1     5
2     5
3     5
4     4
     ..
62    1
63    1
64    1
65    1
66    1
Name: relevancy, Length: 67, dtype: int64

And now we build the test set which is the training set but disordered at random

In [12]:
data_random_order = data.sample(frac=1)
data_random_order

,loinc_num,long_common_name,component,system,property,relevancy
6,1003-3,Indirect antiglobulin test.complement specific...,Indirect antiglobulin test.complement specific...,Ser/Plas,ACnc,2
10,1751-7,Albumin [Mass/volume] in Serum or Plasma,Albumin,Ser/Plas,MCnc,2
48,1798-8,Amylase [Enzymatic activity/volume] in Serum o...,Amylase,Ser/Plas,CCnc,1
38,6768-6,Alkaline phosphatase [Enzymatic activity/volum...,Alkaline phosphatase,Ser/Plas,CCnc,2
23,2039-6,Carcinoembryonic Ag [Mass/volume] in Serum or ...,Carcinoembryonic Ag,Ser/Plas,MCnc,2
...,...,...,...,...,...,...
11,1759-0,Albumin/Globulin [Mass ratio] in Serum or Plasma,Albumin/Globulin,Ser/Plas,MCrto,2
41,890-4,Blood group antibody screen [Presence] in Seru...,Blood group antibody screen,Ser/Plas,ACnc,2
12,17861-6,Calcium [Mass/volume] in Serum or Plasma,Calcium,Ser/Plas,MCnc,2
33,26478-8,Lymphocytes/100 leukocytes in Blood,Lymphocytes/100 leukocytes,Bld,NFr,2


In [13]:
y_test= data_random_order.relevancy
X_test= data_random_order.drop('relevancy',axis=1)

We do the same thing for the val set

In [14]:
data_random_order = data.sample(frac=1)
data_random_order

y_val= data_random_order.relevancy
X_val= data_random_order.drop('relevancy',axis=1)

We need to convert the input data to category in order the algorithm to work

In [15]:
X_train = X_train.astype('category')
X_test = X_test.astype('category')
X_val = X_val.astype('category')

In [16]:
query_train = [X_train.shape[0]]
query_test = [X_test.shape[0]]
query_val = [X_val.shape[0]]

In [24]:
query_test

[67]

## Modeling

In [163]:
import pandas as pd
pd.options.display.max_columns = None
df = pd.read_excel('loinc.xlsx',sheet_name=None)
white=df['White blood cells count']
bilirubin=df['bilirubin in plasma']
glucose=df['glucose in blood']

list_of_names_1 = white['long_common_name'].to_list()
list_of_rank_white = white['rank'].to_list()
list_of_rank_bilirubin = bilirubin['rank'].to_list()
list_of_rank_glucose = glucose['rank'].to_list()
list_of_names_2 = white['long_common_name'].to_list()
list_of_names_3 = white['long_common_name'].to_list()




In [56]:
query_1 = "White blood cells count"
query_2="bilirubin in plasma"
query_3 ="glucose in blood"

In [57]:
import tensorflow as tf

relevance_grades = tf.constant([
    list_of_rank_white,
    list_of_rank_bilirubin,
    list_of_rank_glucose
])
relevance_grades

<tf.Tensor: shape=(3, 67), dtype=int32, numpy=
array([[ 6,  7,  5,  0,  0,  7,  7,  0,  4,  7,  0,  6,  7,  7, 10,  6,
         9,  7,  9,  7,  0,  8,  6,  8,  9,  7,  7,  7,  6,  3,  7,  8,
         5,  9,  7,  0,  7,  0,  7,  7,  7,  7,  7,  0,  7,  7,  7,  7,
         6,  0,  7,  7,  8,  7,  0,  8,  7,  8,  9,  0,  8,  8,  9,  7,
         0,  7,  0],
       [ 7,  4,  9,  8,  7,  2,  3,  9,  5,  8,  5,  6,  9,  4,  2,  8,
        10,  5,  1,  4,  2,  9,  3, 10,  5, 10,  6, 10,  6,  6,  1,  3,
         1,  8,  2,  2, 10,  8,  3,  3,  8,  6,  4,  5,  9,  5,  2,  2,
         6,  7,  4,  2, 10,  5,  1,  1,  6,  6,  9,  9,  7,  3,  7, 10,
         4,  4,  4],
       [ 3,  6,  3,  1,  5,  4,  1,  4,  4,  6, 10,  3,  2,  3, 10,  7,
         9,  3,  6,  1,  7,  9, 10,  6,  3,  7,  2,  2,  9,  9,  9, 10,
         9,  7,  2,  1,  4,  9,  4,  4, 10,  3, 10,  1,  2,  7,  1,  2,
         2,  8,  6,  3,  8, 10,  4,  8,  8,  8,  9,  8,  2,  2,  9, 10,
         6, 10,  6]], dtype=int32)>

In [165]:
combined_texts = [query_1, *list_of_names_1,query_2, *list_of_names_2,query_3, *list_of_names_3]
tokeniser = tf.keras.preprocessing.text.Tokenizer()
tokeniser.fit_on_texts(combined_texts)

# we add one here to account for the padding word
vocab_size = max(tokeniser.index_word) + 1
print(vocab_size)

129


In [166]:
for idx, word in tokeniser.index_word.items():
    print(f"index {idx} - {word}")

index 1 - in
index 2 - volume
index 3 - plasma
index 4 - serum
index 5 - or
index 6 - blood
index 7 - mass
index 8 - moles
index 9 - susceptibility
index 10 - glucose
index 11 - bilirubin
index 12 - total
index 13 - type
index 14 - calcium
index 15 - presence
index 16 - enzymatic
index 17 - activity
index 18 - by
index 19 - product
index 20 - group
index 21 - albumin
index 22 - aminotransferase
index 23 - abo
index 24 - protein
index 25 - c
index 26 - indirect
index 27 - unspecified
index 28 - specimen
index 29 - fluid
index 30 - specific
index 31 - cholesterol
index 32 - leukocytes
index 33 - ionized
index 34 - unit
index 35 - chloride
index 36 - choriogonadotropin
index 37 - lymphocytes
index 38 - 100
index 39 - carbon
index 40 - dioxide
index 41 - reactive
index 42 - rh
index 43 - ampicillin
index 44 - corrected
index 45 - for
index 46 - synovial
index 47 - other
index 48 - antibiotic
index 49 - vancomycin
index 50 - aspartate
index 51 - cefazolin
index 52 - hepatitis
index 53 - b
i

In [167]:
EMBEDDING_DIMS = 2
import numpy as np

embeddings = np.random.randn(vocab_size, EMBEDDING_DIMS).astype(np.float32)


In [168]:
query_1_embedding_index = tokeniser.texts_to_sequences([query_1])
query_1_embeddings = np.array([embeddings[x] for x in query_1_embedding_index])
query_1_embeddings_avg = tf.reduce_mean(query_1_embeddings, axis=1, keepdims=True).numpy()

print(query_1_embeddings_avg)


[[[-0.5350472   0.64500546]]]


In [169]:
query_2_embedding_index = tokeniser.texts_to_sequences([query_2])
query_2_embeddings = np.array([embeddings[x] for x in query_2_embedding_index])
query_2_embeddings_avg = tf.reduce_mean(query_2_embeddings, axis=1, keepdims=True).numpy()

print(query_2_embeddings_avg)


[[[-0.569122    0.32183835]]]


In [170]:
query_3_embedding_index = tokeniser.texts_to_sequences([query_3])
query_3_embeddings = np.array([embeddings[x] for x in query_3_embedding_index])
query_3_embeddings_avg = tf.reduce_mean(query_3_embeddings, axis=1, keepdims=True).numpy()

print(query_3_embeddings_avg)

[[[0.44634297 0.29739067]]]


In [172]:
query_embeddings = np.row_stack([query_1_embeddings_avg, query_2_embeddings_avg,query_3_embeddings_avg])
print(query_embeddings)


[[[-0.5350472   0.64500546]]

 [[-0.569122    0.32183835]]

 [[ 0.44634297  0.29739067]]]


In [173]:
docs_sequences = []
for docs_list in [list_of_names_1,list_of_names_2,list_of_names_3]:
    docs_sequences.append(tokeniser.texts_to_sequences(docs_list))

docs_embeddings = []
for docs_set in docs_sequences:
    this_docs_set = []
    for doc in docs_set:
        this_doc_embeddings = np.array([embeddings[idx] for idx in doc])
        this_docs_set.append(this_doc_embeddings)
    docs_embeddings.append(this_docs_set)

In [174]:
for embeddings in docs_embeddings[0]:
    print()
    print(embeddings)


[[-0.94912016  0.4888725 ]
 [ 1.3922724  -1.2864048 ]
 [-0.13748859 -0.03609164]
 [ 0.01620915  0.5633759 ]
 [ 0.64662874  0.7605802 ]
 [-2.0997694   0.7952226 ]]

[[-0.85470283 -0.2715331 ]
 [ 0.42086697 -0.64820355]
 [-0.82032037 -0.2921808 ]
 [ 0.89296436 -1.9160159 ]
 [ 1.4176027   0.8966453 ]
 [-0.15749206 -0.24092545]
 [ 0.01620915  0.5633759 ]
 [ 0.29102513  1.0587444 ]
 [-0.5940916  -1.3826854 ]
 [-0.77445495 -0.08673337]]

[[-0.94912016  0.4888725 ]
 [ 1.3922724  -1.2864048 ]
 [-0.61178464 -1.50824   ]
 [-0.82032037 -0.2921808 ]
 [ 0.01620915  0.5633759 ]
 [-1.2266282   0.27917525]
 [-2.1577234  -0.6861356 ]]

[[ 0.09114286  0.4092208 ]
 [-0.17993607  0.39694065]
 [ 1.964589    0.5775452 ]]

[[-0.03791303 -0.27078316]
 [ 1.964589    0.5775452 ]]

[[ 0.40739354  0.18938231]
 [ 0.42086697 -0.64820355]
 [-0.82032037 -0.2921808 ]
 [ 0.01620915  0.5633759 ]
 [ 0.29102513  1.0587444 ]
 [-0.5940916  -1.3826854 ]
 [-0.77445495 -0.08673337]]

[[ 0.14176753 -1.1884828 ]
 [-1.4304818   

In [175]:
for embeddings in docs_embeddings[1]:
    print()
    print(embeddings)


[[-0.94912016  0.4888725 ]
 [ 1.3922724  -1.2864048 ]
 [-0.13748859 -0.03609164]
 [ 0.01620915  0.5633759 ]
 [ 0.64662874  0.7605802 ]
 [-2.0997694   0.7952226 ]]

[[-0.85470283 -0.2715331 ]
 [ 0.42086697 -0.64820355]
 [-0.82032037 -0.2921808 ]
 [ 0.89296436 -1.9160159 ]
 [ 1.4176027   0.8966453 ]
 [-0.15749206 -0.24092545]
 [ 0.01620915  0.5633759 ]
 [ 0.29102513  1.0587444 ]
 [-0.5940916  -1.3826854 ]
 [-0.77445495 -0.08673337]]

[[-0.94912016  0.4888725 ]
 [ 1.3922724  -1.2864048 ]
 [-0.61178464 -1.50824   ]
 [-0.82032037 -0.2921808 ]
 [ 0.01620915  0.5633759 ]
 [-1.2266282   0.27917525]
 [-2.1577234  -0.6861356 ]]

[[ 0.09114286  0.4092208 ]
 [-0.17993607  0.39694065]
 [ 1.964589    0.5775452 ]]

[[-0.03791303 -0.27078316]
 [ 1.964589    0.5775452 ]]

[[ 0.40739354  0.18938231]
 [ 0.42086697 -0.64820355]
 [-0.82032037 -0.2921808 ]
 [ 0.01620915  0.5633759 ]
 [ 0.29102513  1.0587444 ]
 [-0.5940916  -1.3826854 ]
 [-0.77445495 -0.08673337]]

[[ 0.14176753 -1.1884828 ]
 [-1.4304818   

In [176]:
for embeddings in docs_embeddings[2]:
    print()
    print(embeddings)


[[-0.94912016  0.4888725 ]
 [ 1.3922724  -1.2864048 ]
 [-0.13748859 -0.03609164]
 [ 0.01620915  0.5633759 ]
 [ 0.64662874  0.7605802 ]
 [-2.0997694   0.7952226 ]]

[[-0.85470283 -0.2715331 ]
 [ 0.42086697 -0.64820355]
 [-0.82032037 -0.2921808 ]
 [ 0.89296436 -1.9160159 ]
 [ 1.4176027   0.8966453 ]
 [-0.15749206 -0.24092545]
 [ 0.01620915  0.5633759 ]
 [ 0.29102513  1.0587444 ]
 [-0.5940916  -1.3826854 ]
 [-0.77445495 -0.08673337]]

[[-0.94912016  0.4888725 ]
 [ 1.3922724  -1.2864048 ]
 [-0.61178464 -1.50824   ]
 [-0.82032037 -0.2921808 ]
 [ 0.01620915  0.5633759 ]
 [-1.2266282   0.27917525]
 [-2.1577234  -0.6861356 ]]

[[ 0.09114286  0.4092208 ]
 [-0.17993607  0.39694065]
 [ 1.964589    0.5775452 ]]

[[-0.03791303 -0.27078316]
 [ 1.964589    0.5775452 ]]

[[ 0.40739354  0.18938231]
 [ 0.42086697 -0.64820355]
 [-0.82032037 -0.2921808 ]
 [ 0.01620915  0.5633759 ]
 [ 0.29102513  1.0587444 ]
 [-0.5940916  -1.3826854 ]
 [-0.77445495 -0.08673337]]

[[ 0.14176753 -1.1884828 ]
 [-1.4304818   

In [177]:
docs_averaged_embeddings = []
for docs_set in docs_embeddings:
    this_docs_set = []
    for doc in docs_set:
        this_docs_set.append(tf.reduce_mean(doc, axis=0, keepdims=True))
    concatenated_docs_set = tf.concat(this_docs_set, axis=0).numpy()
    docs_averaged_embeddings.append(concatenated_docs_set)
    
docs_averaged_embeddings = np.array(docs_averaged_embeddings)

In [178]:
print(docs_averaged_embeddings)

[[[-1.88544646e-01  2.14259103e-01]
  [-1.62393507e-02 -2.31951192e-01]
  [-6.22442186e-01 -3.48791063e-01]
  [ 6.25265300e-01  4.61235553e-01]
  [ 9.63337958e-01  1.53381035e-01]
  [-1.50481731e-01 -8.54715109e-02]
  [-4.26483959e-01  2.77833119e-02]
  [ 1.01347029e+00  2.80928165e-01]
  [-1.33722257e-02 -2.58865785e-02]
  [-3.78701359e-01 -2.69806385e-01]
  [ 3.54667343e-02  1.61625240e-02]
  [ 2.59102136e-01  4.59528342e-02]
  [-2.12887958e-01 -4.06205475e-01]
  [-3.70017320e-01 -6.93099871e-02]
  [ 6.54875338e-01  2.65926749e-01]
  [ 2.91296184e-01  1.54019505e-01]
  [-4.96698022e-01 -6.51858672e-02]
  [-3.61828804e-01  1.72968715e-01]
  [ 5.39026499e-01 -2.58626968e-01]
  [-2.56283164e-01 -3.05656433e-01]
  [ 1.70826721e+00  5.10015428e-01]
  [ 1.79597229e-01  8.44390094e-02]
  [-1.43124908e-03 -1.10338680e-01]
  [-5.18708229e-01 -1.03213668e-01]
  [ 1.87146619e-01  2.40757018e-01]
  [-6.01616651e-02 -1.13477334e-01]
  [-4.78302896e-01 -2.74178922e-01]
  [-1.51030645e-01  9.338528

In [179]:
print(docs_averaged_embeddings.shape)

(3, 67, 2)


In [182]:
 NUM_DOCS_PER_QUERY = 67

expanded_queries = tf.gather(query_embeddings, [0 for x in range(NUM_DOCS_PER_QUERY)], axis=1).numpy()

print(type(query_embeddings))
print(expanded_queries.shape)


<class 'numpy.ndarray'>
(3, 67, 2)


In [181]:
expanded_batch = np.concatenate([expanded_queries, docs_averaged_embeddings], axis=-1)

print(expanded_batch)

[[[-5.35047174e-01  6.45005465e-01 -1.88544646e-01  2.14259103e-01]
  [-5.35047174e-01  6.45005465e-01 -1.62393507e-02 -2.31951192e-01]
  [-5.35047174e-01  6.45005465e-01 -6.22442186e-01 -3.48791063e-01]
  [-5.35047174e-01  6.45005465e-01  6.25265300e-01  4.61235553e-01]
  [-5.35047174e-01  6.45005465e-01  9.63337958e-01  1.53381035e-01]
  [-5.35047174e-01  6.45005465e-01 -1.50481731e-01 -8.54715109e-02]
  [-5.35047174e-01  6.45005465e-01 -4.26483959e-01  2.77833119e-02]
  [-5.35047174e-01  6.45005465e-01  1.01347029e+00  2.80928165e-01]
  [-5.35047174e-01  6.45005465e-01 -1.33722257e-02 -2.58865785e-02]
  [-5.35047174e-01  6.45005465e-01 -3.78701359e-01 -2.69806385e-01]
  [-5.35047174e-01  6.45005465e-01  3.54667343e-02  1.61625240e-02]
  [-5.35047174e-01  6.45005465e-01  2.59102136e-01  4.59528342e-02]
  [-5.35047174e-01  6.45005465e-01 -2.12887958e-01 -4.06205475e-01]
  [-5.35047174e-01  6.45005465e-01 -3.70017320e-01 -6.93099871e-02]
  [-5.35047174e-01  6.45005465e-01  6.54875338e-

### AdaRank definition

In [25]:
import numpy as np
import re
import sys


def group_counts(arr):
    d = np.ones(arr.size, dtype=int)
    d[1:] = (arr[:-1] != arr[1:]).astype(int)
    return np.diff(np.where(np.append(d, 1))[0])


def group_offsets(arr):
    """Return a sequence of start/end offsets for the value subgroups in the input"""
    d = np.ones(arr.size, dtype=int)
    d[1:] = (arr[:-1] != arr[1:]).astype(int)
    idx = np.where(np.append(d, 1))[0]
    return zip(idx, idx[1:])


def load_docno(fname, letor=False):
    """Load docnos from the input in the SVMLight format"""
    if letor:
        docno_pattern = re.compile(r'#\s*docid\s*=\s*(\S+)')
    else:
        docno_pattern = re.compile(r'#\s*(\S+)')

    docno = []
    for line in open(fname):
        if line.startswith('#'):
            continue
        m = re.search(docno_pattern, line)
        if m is not None:
            docno.append(m.group(1))
    return np.array(docno)


def print_trec_run(qid, docno, pred, run_id='exp', output=None):
    """Print TREC-format run to output"""
    if output is None:
        output = sys.stdout
    for a, b in group_offsets(qid):
        idx = np.argsort(-pred[a:b]) + a  # note the minus and plus a
        for rank, i in enumerate(idx, 1):
            output.write('{qid} Q0 {docno} {rank} {sim} {run_id}\n'.
                         format(qid=qid[i], docno=docno[i], rank=rank, sim=pred[i], run_id=run_id))

In [26]:
from __future__ import division

import numpy as np

class Scorer(object):
    def __init__(self, score_func, **kwargs):
        self.score_func = score_func
        self.kwargs = kwargs

    def __call__(self, *args):
        return self.score_func(*args, **self.kwargs)


# Precision
#
def _p_score(y_true, y_pred, k=None):
    order = np.argsort(-y_pred)
    y_true = np.take(y_true, order[:k])
    return np.sum(y_true > 0) / len(y_true)


def p_score(y_true, y_pred, qid, k=None):
    return np.array([_p_score(y_true[a:b], y_pred[a:b], k=k) for a, b in group_offsets(qid)])


class PScorer(Scorer):
    def __init__(self, **kwargs):
        super(PScorer, self).__init__(p_score, **kwargs)


# AP (Average Precision)
#
def _ap_score(y_true, y_pred):
    order = np.argsort(-y_pred)
    y_true = np.take(y_true, order)
    pos = 1 + np.where(y_true > 0)[0]
    n_rels = 1 + np.arange(len(pos))
    return np.mean(n_rels / pos) if len(pos) > 0 else 0


def ap_score(y_true, y_pred, qid):
    return np.array([_ap_score(y_true[a:b], y_pred[a:b]) for a, b in group_offsets(qid)])


class APScorer(Scorer):
    def __init__(self):
        super(APScorer, self).__init__(ap_score)


# DCG/nDCG (Normalized Discounted Cumulative Gain)
#
def _burges_dcg(y_true, y_pred, k=None):
    # order = np.argsort(y_pred)[::-1]
    order = np.argsort(-y_pred)
    y_true = np.take(y_true, order[:k])
    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(gain)) + 2)
    return np.sum(gain / discounts)


def _trec_dcg(y_true, y_pred, k=None):
    order = np.argsort(-y_pred)
    y_true = np.take(y_true, order[:k])
    gain = y_true
    discounts = np.log2(np.arange(len(gain)) + 2)
    return np.sum(gain / discounts)


def _dcg_score(y_true, y_pred, qid, k=None, dcg_func=None):
    assert dcg_func is not None
    y_true = np.maximum(y_true, 0)
    return np.array([dcg_func(y_true[a:b], y_pred[a:b], k=k) for a, b in group_offsets(qid)])


def _ndcg_score(y_true, y_pred, qid, k=None, dcg_func=None):
    assert dcg_func is not None
    y_true = np.maximum(y_true, 0)
    dcg = _dcg_score(y_true, y_pred, qid, k=k, dcg_func=dcg_func)
    idcg = np.array([dcg_func(np.sort(y_true[a:b]), np.arange(0, b - a), k=k)
                     for a, b in group_offsets(qid)])
    assert (dcg <= idcg).all()
    idcg[idcg == 0] = 1
    return dcg / idcg


def dcg_score(y_true, y_pred, qid, k=None, version='burges'):
    assert version in ['burges', 'trec']
    dcg_func = _burges_dcg if version == 'burges' else _trec_dcg
    return _dcg_score(y_true, y_pred, qid, k=k, dcg_func=dcg_func)


def ndcg_score(y_true, y_pred, qid, k=None, version='burges'):
    assert version in ['burges', 'trec']
    dcg_func = _burges_dcg if version == 'burges' else _trec_dcg
    return _ndcg_score(y_true, y_pred, qid, k=k, dcg_func=dcg_func)


class DCGScorer(Scorer):
    def __init__(self, **kwargs):
        super(DCGScorer, self).__init__(dcg_score, **kwargs)


class NDCGScorer(Scorer):
    def __init__(self, **kwargs):
        super(NDCGScorer, self).__init__(ndcg_score, **kwargs)

In [27]:
from __future__ import print_function, division

import math
import numpy as np
import sklearn
import sys

from sklearn.utils import check_X_y


class AdaRank(sklearn.base.BaseEstimator):
    """AdaRank algorithm"""

    def __init__(self, max_iter=500, tol=0.0001, estop=1, verbose=False, scorer=None):
        self.max_iter = max_iter
        self.tol = tol
        self.estop = estop
        self.verbose = verbose
        self.scorer = scorer

    def fit(self, X, y, qid, X_valid=None, y_valid=None, qid_valid=None):
        """Fit a model to the data"""
        X, y = check_X_y(X, y, 'csr')
        X = X.toarray()

        if X_valid is None:
            X_valid, y_valid, qid_valid = X, y, qid
        else:
            X_valid, y_valid = check_X_y(X_valid, y_valid, 'csr')
            X_valid = X_valid.toarray()

        n_queries = np.unique(qid).shape[0]
        weights = np.ones(n_queries, dtype=np.float64) / n_queries
        weak_rankers = []
        coef = np.zeros(X.shape[1])

        # use nDCG@10 as the default scorer
        if self.scorer is None:
            self.scorer = NDCGScorer(k=10)

        # precompute performance measurements for all weak rankers
        weak_ranker_score = []
        for j in range(X.shape[1]):
            pred = X[:, j].ravel()
            weak_ranker_score.append(self.scorer(y, pred, qid))

        best_perf_train = -np.inf
        best_perf_valid = -np.inf
        used_fids = []
        estop = None

        self.n_iter = 0
        while self.n_iter < self.max_iter:
            self.n_iter += 1

            best_weighted_average = -np.inf
            best_weak_ranker = None
            for fid, score in enumerate(weak_ranker_score):
                if fid in used_fids:
                    continue
                weighted_average = np.dot(weights, score)
                if weighted_average > best_weighted_average:
                    best_weak_ranker = {'fid': fid, 'score': score}
                    best_weighted_average = weighted_average

            # stop when all the weaker rankers are out
            if best_weak_ranker is None:
                break

            h = best_weak_ranker
            h['alpha'] = 0.5 * (math.log(np.dot(weights, 1 + h['score']) /
                                         np.dot(weights, 1 - h['score'])))
            weak_rankers.append(h)

            # update the ranker
            coef[h['fid']] += h['alpha']

            # if len(used_fids) > 5:
            #     used_fids.pop(0)
            # used_fids.append(h['fid'])

            # score both training and validation data
            score_train = self.scorer(y, np.dot(X, coef), qid)
            perf_train = score_train.mean()

            perf_valid = perf_train
            if X_valid is not X:
                perf_valid = self.scorer(y_valid, np.dot(X_valid, coef), qid_valid).mean()

            if self.verbose:
                print('{n_iter}\t{alpha}\t{fid}\t{score}\ttrain {train:.4f}\tvalid {valid:.4f}'.
                      format(n_iter=self.n_iter, alpha=h['alpha'], fid=h['fid'],
                             score=h['score'][:5], train=perf_train, valid=perf_valid),
                      file=sys.stderr)

            # update the best validation scores
            if perf_valid > best_perf_valid + self.tol:
                estop = 0
                best_perf_valid = perf_valid
                self.coef_ = coef.copy()
            else:
                estop += 1

            # update the best training score
            if perf_train > best_perf_train + self.tol:
                best_perf_train = perf_train
            else:
                # stop if scores on both sets fail to improve
                if estop >= self.estop:
                    break

            # update weights
            new_weights = np.exp(-score_train)
            weights = new_weights / new_weights.sum()

        return self

    def predict(self, X, qid):
        """Make predictions"""
        return np.dot(X.toarray(), self.coef_)

### AdaRank fitting and testing

In [28]:
scorer = NDCGScorer(k=10)
model = AdaRank(max_iter=100, estop=10, scorer=NDCGScorer(k=5))
model = model.fit(docs_averaged_embeddings, y_train, query_train)
pred = model.predict(X_test, query_test)
print(scorer(y_test, pred, query_test).mean())

ValueError: ignored

In [ ]:
gbm.fit(X_train, y_train, group=query_train,
        eval_set=[(X_val, y_val)], eval_group=[query_val],
        eval_at=[5, 10, 20, 40], early_stopping_rounds=50)

In [ ]:
test_pred = gbm.predict(X_test)
test_pred

In [ ]:
X_test["predicted_ranking"] = test_pred

X_test.sort_values("predicted_ranking", ascending=False)

